# SQLgen 

Generate SQL query 
- for `Postgres`
- on-prem using `Ollama` with `IBM Granite Code` model 

### Replace database schema and user prompt 

In [26]:
model = 'granite-code:8b-instruct'

schema = """  
Customers:  
- customer_id (INT, PRIMARY KEY)  
- first_name (VARCHAR)  
- last_name (VARCHAR)  
- email (VARCHAR)  
- phone (VARCHAR)  
- address (VARCHAR)  
- city (VARCHAR)  
- state (VARCHAR)  
- zip_code (VARCHAR)  
  
Products:  
- product_id (INT, PRIMARY KEY)  
- product_name (VARCHAR)  
- description (TEXT)  
- category (VARCHAR)  
- price (DECIMAL)  
- stock_quantity (INT)  
  
Orders:  
- order_id (INT, PRIMARY KEY)  
- customer_id (INT, FOREIGN KEY REFERENCES Customers)  
- order_date (DATE)  
- total_amount (DECIMAL)  
- status (VARCHAR)  
  
Order_Items:  
- order_item_id (INT, PRIMARY KEY)  
- order_id (INT, FOREIGN KEY REFERENCES Orders)  
- product_id (INT, FOREIGN KEY REFERENCES Products)  
- quantity (INT)  
- price (DECIMAL)  
  
Reviews:  
- review_id (INT, PRIMARY KEY)  
- product_id (INT, FOREIGN KEY REFERENCES Products)  
- customer_id (INT, FOREIGN KEY REFERENCES Customers)  
- rating (INT)  
- comment (TEXT)  
- review_date (DATE)  
  
Employees:  
- employee_id (INT, PRIMARY KEY)  
- first_name (VARCHAR)  
- last_name (VARCHAR)  
- email (VARCHAR)  
- phone (VARCHAR)  
- hire_date (DATE)  
- job_title (VARCHAR)  
- department (VARCHAR)  
- salary (DECIMAL)  
"""

user_prompt = """
Get the list of customers who have placed orders but have not provided any reviews, along with the total amount they have spent on orders.
"""

### Gen! (and scroll to bottom)

In [27]:
%pip install -U langchain langchain-ollama 

# set system prompt for Ollama
system_prompt = f"""
Transform the following natural language requests into valid Postgres SQL queries. Assume a database with the following tables and columns exists: 
{schema}
Provide the SQL query that would retrieve the data based on the natural language request.
"""

# pass the prompts to Ollama and get the response
from langchain_ollama import ChatOllama
from langchain_core.messages import AIMessage
from IPython.display import Markdown


def generate_response(system_prompt, user_prompt, model):
    llm = ChatOllama(
            model=model,
            temperature=0.8,
            num_predict=256,
    )
    messages = [
        ("system", system_prompt),
        ("human", user_prompt),
    ]
    return llm.invoke(messages)

response = generate_response(system_prompt, user_prompt, model)
response_content = response.content
Markdown(f"## SQL query generated: \n{response_content}")


Note: you may need to restart the kernel to use updated packages.


## SQL query generated: 
Here is the SQL query to retrieve the list of customers who have placed orders but have not provided any reviews, along with the total amount they have spent on orders:

```sql
SELECT Customers.customer_id, Customers.first_name, Customers.last_name, SUM(Orders.total_amount) AS total_spent
FROM Customers
JOIN Orders ON Customers.customer_id = Orders.customer_id
LEFT JOIN Reviews ON Customers.customer_id = Reviews.customer_id
WHERE Reviews.review_id IS NULL
GROUP BY Customers.customer_id, Customers.first_name, Customers.last_name
```

This query uses a LEFT JOIN to include customers who have not provided any reviews in the result set. The WHERE clause filters out customers with a review ID, and the GROUP BY clause groups the results by customer ID, first name, and last name. The SUM function calculates the total amount spent by each customer on orders.
